In [2]:
import setup, os, json
from recordings.audio_file_path import audio_file_path,transcript_name
import librosa
import numpy as np
y, sr = librosa.load(audio_file_path)

# Convert to desired data type (float32 or float64)
audio_array = y.astype(np.float32)  # or np.float64

# Ensure it's 1D array
if audio_array.ndim > 1:
    audio_array = audio_array.flatten()

print(f"Shape: {audio_array.shape}, Data type: {audio_array.dtype}, Sampling rate: {sr}")

transcribing file: 
/Users/ayserchoudhury/playground/jarvis/transcribe/recordings/sample.wav
Shape: (860803,), Data type: float32, Sampling rate: 22050


In [4]:
from transformers import pipeline
pipe = pipeline("automatic-speech-recognition",model="openai/whisper-large-v3",max_new_tokens=100, chunk_length_s=30)

from pyannote.audio import Pipeline
from keys import pyannote_key

diarization_pipeline = pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=pyannote_key)


# returns diarization info such as segment start and end times, and speaker id
def diarization_info(res):
    starts = []
    ends = []
    speakers = []

    for segment, _, speaker in res.itertracks(yield_label=True):
        starts.append(segment.start)
        ends.append(segment.end)
        speakers.append(speaker)

    return starts, ends, speakers


# plot diarization results on a graph
def plot_diarization(starts, ends, speakers):
    fig = go.Figure()

    # Define a color map for different speakers
    num_speakers = len(set(speakers))
    colors = [f"hsl({h},80%,60%)" for h in np.linspace(0, 360, num_speakers)]

    # Plot each segment with its speaker's color
    for start, end, speaker in zip(starts, ends, speakers):
        speaker_id = list(set(speakers)).index(speaker)
        fig.add_trace(
            go.Scatter(
                x=[start, end],
                y=[speaker_id, speaker_id],
                mode="lines",
                line=dict(color=colors[speaker_id], width=15),
                showlegend=False,
            )
        )

    fig.update_layout(
        title="Speaker Diarization",
        xaxis=dict(title="Time"),
        yaxis=dict(title="Speaker"),
        height=600,
        width=800,
    )

    return fig


def transcribe(sr, data):
    processed_data = np.array(data).astype(np.float32) / 32767.0

    # results from the pipeline
    transcription_res = pipe({"sampling_rate": sr, "raw": processed_data})["text"]

    return transcription_res


def transcribe_diarize(audio):
    sr, data = audio
    processed_data = np.array(data).astype(np.float32) / 32767.0
    waveform_tensor = torch.tensor(processed_data[np.newaxis, :])

    transcription_res = transcribe(sr, data)

    # results from the diarization pipeline
    diarization_res = diarization_pipeline(
        {"waveform": waveform_tensor, "sample_rate": sr}
    )

    # Get diarization information
    starts, ends, speakers = diarization_info(diarization_res)

    # results from the transcription pipeline
    diarized_transcription = ""

    # Get transcription results for each speaker segment
    for start_time, end_time, speaker_id in zip(starts, ends, speakers):
        segment = data[int(start_time * sr) : int(end_time * sr)]
        diarized_transcription += f"{speaker_id} {round(start_time, 2)}:{round(end_time, 2)} \t {transcribe(sr, segment)}\n"

    # Plot diarization
    diarization_plot = plot_diarization(starts, ends, speakers)

    return transcription_res, diarized_transcription, diarization_plot


/Users/ayserchoudhury/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Users/ayserchoudhury/anaconda3/lib/python3.11/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [5]:
x = transcribe_diarize(audio_array)
'''
Look into the GR.Audio file and see what the expected return is there
'''

ValueError: too many values to unpack (expected 2)

In [ ]:
print(type(x))
print(x)